## Machine locale


In [1]:
#cd /srv/Annif/anaconda3/bin
#nohup ./jupyter-notebook --no-browser --port=8080 --NotebookApp.allow_origin='*' --NotebookApp.port_retries=0 --NotebookApp.token='toto'  --NotebookApp.disable_check_xsrf=True --NotebookApp.allow_remote_access=True &
#lancer le serveur web pour la page app.py   :./uvicorn app:app --reload
#uvicorn app:app --port 5000
#cd /var/lib/docker/Annif/anaconda3/bin
#lancer jupyter : nohup ./jupyter-notebook --no-browser --ip=10.34.202.165 --port=8080 --NotebookApp.allow_origin='*' --NotebookApp.port_retries=0 --NotebookApp.token='toto' --NotebookApp.allow_remote_access=True &
#lancer les webservices : gunicorn --worker-class uvicorn.workers.UvicornWorker --bind '10.34.202.165:8000' --daemon app:app
#installer des appli dans conda : ./conda install -c anaconda cx_Oracle
import os
path = "/srv/Annif/CARAML"
os.chdir(path)
os.listdir()
data_path = path + "\\data"
input_path = path + "\\input"
output_path = path + "\\output"
others_path = path + "\\others"

#Importation de librairies

In [2]:
!python3 -m spacy download fr_core_news_sm

2022-01-24 12:14:36.796564: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-24 12:14:36.796690: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "/usr/lib64/python3.6/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib64/python3.6/runpy.py", line 142, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib64/python3.6/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/srv/Annif/.local/lib/python3.6/site-packages/spacy/__init__.py", line 11, in <module>
    from thinc.api import prefer_gpu, require_gpu, require_cpu  # noqa: F401
  File "/srv/Annif/.local/lib/python3.6

In [3]:
!pip install scikit-multilearn

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install Flask

Defaulting to user installation because normal site-packages is not writeable


In [5]:
# librairies générales
import pandas as pd
import numpy as np
import csv
import requests
import xml.etree.ElementTree as ET

#cbd 09102021
import cx_Oracle
import os 

# librairie affichage
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(font_scale=1)

#Pour le prétratiement
import spacy
import fr_core_news_sm
#import en_core_web_sm

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop
import re
import unicodedata
from nltk.stem.snowball import SnowballStemmer

#train test split
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split

#Tf Idf
from sklearn.feature_extraction.text import TfidfVectorizer

# OnevsRest
from sklearn.multiclass import OneVsRestClassifier

#Encodage des labels
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()


# Models
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.pipeline import Pipeline
import joblib
import pickle

#Evaluation
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score
# from sklearn.metrics import roc_auc_score 
# from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
# from functools import partial
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss

#Validation croisée
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#matrice de confusion
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

#from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
from collections import Counter

seed=30

[nltk_data] Downloading package punkt to /srv/Annif/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /srv/Annif/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# définition des fonctions

In [6]:
def get_dataset(path):
  dataset = pd.read_csv(path, converters={'titre': eval, 'resume': eval, 'codeD':eval})
  return dataset
# sauvegarder un dataframe en csv
def save_dataset_to_csv(df, filename):
  path_destination = 'save/'+ filename + '.csv'
  df.to_csv(path_destination, index=False)
#Importer le dataset préparé
def get_dataset_init(path):
  dataset_init = pd.read_csv(path, converters={'codeD': eval}) 
  return dataset_init
# Lemmatiser les mots (déterminer le lemme) avec Spacy
def lemmatisation(df, lang, save = True):
  df_lem = df.copy()
  if lang == 'french':
    nlp = fr_core_news_sm.load(disable=["parser", "ner"])
  if lang == 'english':
    nlp = en_core_web_sm.load(disable=["parser", "ner"])
  df_lem['titre'] = df_lem['titre'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
  df_lem['resume'] = df_lem['resume'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
  if save:
    save_dataset_to_csv(df_lem, 'dataset_lem')
  return df_lem

# Transformer les phrases en sequences de mot avec word_tokenize.
def tokenizer(df, lang):
  df_token = df.copy()
  df_token.titre = df_token.titre.map(lambda e: word_tokenize(e, language = lang))
  df_token.resume = df_token.resume.map(lambda e: word_tokenize(e, language = lang))
  return df_token

#Les prétraitement suivants permettent de supprimer les ponctuations, de supprimer des stopwords et la lémmatisation. 
#Suppresion des ponctuations (caractères spéciaux)
def funcPonct(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

#Suppression des tokens numeriques
def funcDigit(tokens):
     tokens_p = [word for word in tokens if not word.isdigit()]
     return tokens_p

#Suppresion des stop words 
def funcSW(tokens, lang):
    if lang == 'french':
      stop_words = set(stopwords.words(lang)).union(fr_stop) #concatenation of two french stopword lists
    if lang == 'english':
      stop_words = set(stopwords.words(lang)).union(en_stop) #concatenation of two english stopword lists
    tokens_sw = [w for w in tokens if not w in stop_words]
    return tokens_sw
    
#Encodage des données, les accents seront enlevés
def funcEnc(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

#Stemmatisation (réduire les mots à leurs racines)
def funcSTM(tokens, lang):
    stemmer = SnowballStemmer(language=lang)
    tokens_stm = [stemmer.stem(word) for word in tokens]
    return tokens_stm

#Regrouper les fonctions
def normalize(tokens, lang):
    tokens = funcPonct(tokens)
    tokens = funcDigit(tokens)
    tokens = funcSW(tokens, lang)
    tokens = funcEnc(tokens)
    #tokens = funcSTM(tokens, lang)
    return tokens

#Fonction qui permet d'effectuer tout les pré-traitements au dataset passé en argument
def preprocessing(df, lang, savename, save = False):
  df_pre = df.copy()
  df_pre = lemmatisation(df_pre, lang)
  df_pre = tokenizer(df_pre, lang)
  df_pre.titre = df_pre.titre.map(lambda e: normalize(e, lang))
  df_pre.resume = df_pre.resume.map(lambda e: normalize(e, lang))
  if save:
    save_dataset_to_csv(df_pre, savename)
  return df_pre
# #Convertir liste en set
# def convert_to_tuple(list):
#   return tuple(list)

#Encoder les labels
def encode_label(y,ledomaine,save):
  # remove duplicates from list of idC, then convert the list to string
  y = y.map(np.unique).apply(lambda x: ','.join(map(str, x)))
  #Transformer les labels en dictionnaire : par ex [300, 640] devient {300, 640}
  y = [set(i.split(',')) for i in y]
  #Encodage des labels
  ## on le reinitialise
  mlb = MultiLabelBinarizer()
  y_encoded = mlb.fit_transform(y)
  classes = mlb.classes_
  if save:
      pickle.dump(mlb, open("mlb_"+ledomaine+".pickle", "wb"))
  return y_encoded, classes

#Iterative Splitting pour multilabel
def iterative_train_test_split_dataframe(X, y, test_size):
    df_index = np.expand_dims(X.index.to_numpy(), axis=1)
    X_train, y_train, X_test, y_test = iterative_train_test_split(df_index, y, test_size = test_size)
    X_train = X.loc[X_train[:,0]]
    X_test = X.loc[X_test[:,0]]
    return X_train, y_train, X_test, y_test


#splitting, encoding
def splitting_encoding(df, col_label, test_size,ledomaine, save = True):
  df_traited = df.copy()

  # concatenate tokens
  df_traited['titre'] = df_traited.titre.map(lambda e: ' '.join(e))
  df_traited['resume'] = df_traited.resume.map(lambda e: ' '.join(e)) 
  df_traited['corpus'] = df_traited['titre'] + df_traited['resume']

  # define X and y
  X = df_traited.drop(columns=['idB','titre','resume',col_label])
  y = df_traited[col_label]

  #encode labels
  encoding = encode_label(y,ledomaine, save )
  y = encoding[0]
  classes = encoding[1]

  #Iterative splitting
  X_train_txt, y_train, X_test_txt, y_test = iterative_train_test_split_dataframe(X, y, test_size = test_size)

  return X_train_txt, y_train, X_test_txt, y_test, classes

def vectorizer_tfidf(X_train, X_test, max_df, min_df, max_features, save = True):
  regex_pattern = r'\w{3,}' #des mots qui ont plus de trois caractères
  vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df, max_features=max_features, ngram_range=(1,2), token_pattern=regex_pattern)
  X_train = vectorizer.fit_transform(X_train.corpus)
  X_test = vectorizer.transform(X_test.corpus)
  features = vectorizer.get_feature_names()
  if save:
    pickle.dump(vectorizer, open("tfidf_domaine.pickle", "wb"))
  return X_train, X_test, features

def encoding(df, col_label, ledomaine, save = True):
  df_traited = df.copy()

  # concatenate tokens
  df_traited['titre'] = df_traited.titre.map(lambda e: ' '.join(e))
  df_traited['resume'] = df_traited.resume.map(lambda e: ' '.join(e)) 
  df_traited['corpus'] = df_traited['titre'] + df_traited['resume']

  # define X and y
  X = df_traited.drop(columns=['idB','titre','resume',col_label])
  y = df_traited[col_label]

  #encode labels
  encoding = encode_label(y,ledomaine, save )
  y = encoding[0]
  classes = encoding[1]

  return X, y, classes

def get_concepts_by_domaine(ledomaine, path_dataset, path_conc_dom):
  # lire des fichiers necessaires pour construire le dataset
  dataset_preprocessed = get_dataset(path_dataset)
  idB_idC_codeD_g = pd.read_csv(path_conc_dom)

  # selectionner les documents et les concepts dont le domaine correspond au premier argument 'domaine'
  G = idB_idC_codeD_g.loc[idB_idC_codeD_g['newCodeD'] == ledomaine]
  tmp = pd.merge(dataset_preprocessed, G, on = 'idB')
  # supprimer les documents dont la fréquence de concept dans l'ensemble est inferieur à 3
  tmp = tmp[tmp.groupby('idC').idB.transform(len) > 3]
  tmp_grouped = tmp.groupby(['idB'])['idC'].aggregate(lambda x: tuple(x))
  df_G = pd.merge(dataset_preprocessed, tmp_grouped, on ='idB').drop(columns='codeD')
  return df_G
#Sauvegarder le model 
def save_model(model, modelname, suffix):
  if suffix == 'domaine':
    joblib.dump(model, 'models/'+ modelname + '_' + suffix + '.pkl') 
  else:
    joblib.dump(model, 'models/concept/'+ modelname + '_' + suffix + '.pkl') 

#Entraînement du classifieur passé en argument
def trainning_br(df, X_train, y_train, algo, suffix, save = True):
  if algo == 'lr':
    model = LogisticRegression(solver='lbfgs', max_iter=10000, class_weight='balanced')
  elif algo == 'svc':
    model = LinearSVC(max_iter = 10000, class_weight='balanced')
  elif algo == 'nb':
    model = MultinomialNB()
  elif algo == 'knn':
    model = KNeighborsClassifier(n_neighbors=5)
  elif algo == 'rf':
    model = RandomForestClassifier(random_state=seed, class_weight='balanced')
  else :
    print('The algo ' + algo + ' is not defined!')

  classifier = BinaryRelevance(
    classifier = model
  )
  
  %time classifier.fit(X_train, y_train)#Training the model on dataset
  if save:
    save_model(classifier, algo, suffix)
    
#connexion base oracle
dsn_tns = cx_Oracle.makedsn('orpins-p-scan.v110.abes.fr', '1521', service_name='APISUDOC')

def oracle_connect():
    conn = cx_Oracle.connect(user=r'autorites', password='autorites0ieuf1', dsn=dsn_tns)
    return conn
def oracle_disconnect():
    conn.close() 

# preprocessing des datas et sauvegarde (normalement a ne plus refaire)

In [7]:
ligne a commenter
df = get_dataset_init('input/dataset_final_g.csv')
#df
# formate les entrees
preprocessing(df, 'french', 'dataset_preprocessed')
#preprocessing(df, 'english', dataset_preprocessed_eng')  #Exmple de l'utilisation de la fonction si le dataset est en anglais 

SyntaxError: invalid syntax (<ipython-input-7-02c4ccd02c2a>, line 1)

# Entraînement des modèles (ne plus faire)

### Traitement entrainement

In [8]:
entrainement sur tous les domaines pour les concepts (commenter cette ligne)
new_data_pred = pickle.load(open("pred_domaine.dmp", 'rb'))
new_data_pred.prediction = new_data_pred.prediction.apply(list)

new_data_pred = new_data_pred.explode('prediction')
new_data = pd.read_csv('evaluation/edp_titres_resumes_fr_1.csv')
new_data_preprocessed = preprocessing(new_data, 'french', 'test')
new_classes = new_data_pred.prediction.unique()
pos=0
for i in new_classes:
  pos=pos+1
  if pos<50:
        if i is no  t None:
                if not os.path.isfile('tfidf_' + str(i) + '.pickle') or not os.path.isfile("models/concept/svc_concept_" + str(i) + ".pkl") or not os.path.isfile("mlb_"+str(i)+".pickle"):
                      print(str(i))
                      df_G = get_concepts_by_domaine(str(i), 'save/dataset_preprocessed.csv', 'input/idB_idC_codeD_g.csv')

                      # Copy du dataset dans la variable 'tmp' 
                      tmp = df_G.copy()

                      # Splitting et encoding
                      trait = splitting_encoding(tmp, 'idC', 0.3,str(i), save = True)
                      X_train_txt = trait[0]
                      y_train = trait[1]
                      X_test_txt = trait[2]
                      y_test = trait[3]
                      #classes = trait[4]

                      # TF-IDF Vectorizer
                      regex_pattern = r'\w{3,}' #des mots qui ont plus de trois caractères
                      vectorizer = TfidfVectorizer(max_df = 0.5, min_df = 5, max_features=5000, ngram_range=(1,2), token_pattern=regex_pattern)
                      X_train = vectorizer.fit_transform(X_train_txt.corpus)
                      X_test = vectorizer.transform(X_test_txt.corpus)
                      #features = vectorizer.get_feature_names()
                      ## save tf-idf vectorizer
                      vectorizer_name = "tfidf_" + str(i) + '.pickle'
                      pickle.dump(vectorizer, open(vectorizer_name, "wb"))

                      # Entrainement des models puis la sauvegarde
                      suffixe = 'concept_' + str(i)
                      trainning_br(tmp, X_train, y_train, 'svc', suffixe) 

SyntaxError: invalid syntax (<ipython-input-8-0b3ad79309f9>, line 1)

###Entrainement des modèles

# Prédiction des concepts sur les données de test pour un domaine precis

In [ ]:
domaine = 'G500'
df = get_concepts_by_domaine(domaine, 'save/dataset_preprocessed.csv', 'input/idB_idC_codeD_g.csv')
df
# Copy du dataset dans la variable 'tmp' 
tmp = df.copy()
tmp =df.head(2)
#print(tmp)
# Splitting et encoding
trait = encoding(tmp, 'idC', domaine, save = False)
X_test_txt = trait[0]
classes = trait[2]
vect_name = 'tfidf_' + domaine + '.pickle'
vectorizer = pickle.load(open(vect_name, 'rb'))

X_test = vectorizer.transform(X_test_txt.corpus)
model = joblib.load("models/concept/svc_concept_" + domaine + ".pkl")
y_pred = model.predict(X_test)
#Afficher les résultats de prédiction
mlb = pickle.load(open("mlb_"+domaine+".pickle", 'rb'))
y_pred_inversed = mlb.inverse_transform(y_pred)

for item,label in zip(X_test_txt.corpus, y_pred_inversed):
    print ('%s => %s' % (item[0:15],label))


# prediction des concepts sur les ppn de test

In [15]:
df = get_dataset('save/dataset_preprocessed.csv')
df.shape

(260746, 4)

In [15]:
df.sample(n=10)

,idB,titre,resume,codeD
67292,053669754,"[protection, tiers, face, association, contrib...","[loi, reconner, liberte, public, association, ...","[G320, G320, G100, G320, G300, G320, G320, G320]"
246735,23758610X,"[lapport, logiciel, simulation, algorithme, pr...","[these, objet, etude, apport, introduction, te...","[G510, G621, G510, G621, G510, G370, G621, G37..."
41599,043742459,"[etude, vieillissement, fragilisation, acier, ...","[presente, systeme, indentation, instrumenter,...","[G600, G600, G530, G600, G530]"
159856,158764986,"[vitroceramique, nano, structurer, sio2sno2, f...","[avenement, source, lumineuse, performant, com...","[G600, G530, G530, G540, G600, G530, G540, G54..."
219082,199755973,"[modelisation, dimensionnemer, optimisation, c...","[annee, chiffre, insecurite, routier, montrer,...","[G600, G320, G320, G793, G600, G500]"
98398,087501430,"[spectroscopie, resoudre, temps, laser, blanc,...","[principal, chromophore, endogene, interet, tr...","[G530, G570, G530]"
174014,169770877,"[synthese, analogue, aminoacyl, arnt, conjugue...","[aminoacyl, transferase, fem, catalysent, etap...","[G570, G570, G570]"
211611,194055248,"[modele, resolution, approche, efficace, probl...","[these, interesse, resolution, probleme, optim...","[G510, G510, G621, G510, G510, G621, G510]"
13076,01181568X,"[genetique, population, tilapias, interet, aqu...","[etude, variabilite, genetique, espece, tilapi...","[G570, G570, G540, G570, G570, G910, G570, G63..."
158096,157713725,"[microbiote, intestinal, nouveau, naitre, prem...","[nouveau, naitre, premature, implantation, mic...","[G570, G570, G610, G300, G610, G610, G570, G61..."


#### prediction pour un domaine precis

In [14]:
#test sur un domaine precis
domaine = 'G550'

list_ppn = ['001976508', '001149733']
df_test=df[df.idB.isin(list_ppn)]
df_test['titre'] = df_test.titre.map(lambda e: ' '.join(e))
df_test['resume'] = df_test.resume.map(lambda e: ' '.join(e)) 
df_test['corpus'] = df_test['titre'] + df_test['resume']


vectorizer = pickle.load(open('tfidf_' + domaine + '.pickle', 'rb'))

X_test = vectorizer.transform(df_test.corpus)
model = joblib.load("models/concept/svc_concept_" + domaine + ".pkl")
y_pred = model.predict(X_test)
#Afficher les résultats de prédiction
mlb = pickle.load(open("mlb_"+domaine+".pickle", 'rb'))
y_pred_inversed = mlb.inverse_transform(y_pred)

for item,label in zip(df_test.corpus, y_pred_inversed):
    print ('%s => %s' % (item[0:15],label))

<ipython-input-14-d0b16429b9cb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['titre'] = df_test.titre.map(lambda e: ' '.join(e))
<ipython-input-14-d0b16429b9cb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['resume'] = df_test.resume.map(lambda e: ' '.join(e))
<ipython-input-14-d0b16429b9cb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

modelisation th => ('027253597', '027253783', '027289850', '02735735X', '027392198')
nature genes pr => ('027242722', '027246183', '030900182')


#### sur tous les domaines

In [26]:
#test sur tous les domaines
#recherche les domaines
#attention le 570 n a pas pu etre entrainé car pas assez de données
#domaine = 'G600'
domaine = ''
test=True
test_size=20000

pd.options.mode.chained_assignment = None
list_ppn = ['004762487','005799066','007114621','009380493','011374403','012802689','041456696','041607511','042051266','042415926',
'043801978','043817149','049605976','058637699','059991771','060011165','063462087','067350259','071024166','074137271','078393302',
'083418113','092418945','095140913','109365291','111877016','117378135','120935031','131590553','132481251','138462127','139250905',
'140356533','150214847','152903356','158308123','160615941','161860044','164275193','166019127','166968765','167357921','168875713',
'170131521','172397448','175120447','175707715','178106062','178644463','179799827','180076388','181027356','181848848','183381750',
'185959857','187372837','188126376','188572953','190619694','191269816','196498260','196622883','198270836','202377911','203350073',
'219502250','220219141','223744824','224225146','224824023','225564181','225872048','226775704','227234642','229279023','231745265',
'232378169','232446369','236856871','241151821','242216595','242325602','244376476','244791252','250153025','252824881','11083609X',
'14295456X','16054159X','17080366X','18742070X','22341302X','22348153X','23713392X','24243066X','25417695X']

df = get_dataset('save/dataset_preprocessed.csv')
conn=oracle_connect()
if test:
    # pour le faire sur tous les ppn de test
    dataset_rec = pd.read_csv('/srv/Annif/CARAML/save/ppn_test.csv',usecols=["idB"])
    dataset_test=dataset_rec.head(test_size)
    df_test=df[df.idB.isin(dataset_test.idB)]
else:
    ### sinon sur la liste a commenter dans l'autre cas
    
    ##df.shape
    df_test=df[df.idB.isin(list_ppn)]

df_test.titre = df_test.titre.map(lambda e: ' '.join(e))
df_test.resume = df_test.resume.map(lambda e: ' '.join(e)) 
df_test['corpus'] = df_test.titre + df_test.resume


#df_test.corpus
model = joblib.load("models/svc_domaine.pkl")
mlb = pickle.load(open("mlb_domaine.pickle", 'rb'))
vectorizer = pickle.load(open('tfidf_domaine.pickle', 'rb'))
X_test_vec = vectorizer.transform(df_test.corpus)
y_pred = model.predict(X_test_vec)
#Afficher les résultats de prédiction
y_pred_inversed = mlb.inverse_transform(y_pred)
#for item,label in zip(df_test.idB, y_pred_inversed):
#    print ('%s => %s' % (item[0:15],label))
df_resu_ppn = pd.DataFrame(columns=['idB','indexeur','auto','algo'])
#df_resu_ppn_lst = pd.DataFrame(columns=['Idb','ppn_concept'])
df_resu = pd.DataFrame(columns=['idB','auto'])

df_test['prediction'] = pd.Series(y_pred_inversed, index=df_test.index)
new_data_pred = df_test.explode('prediction')
#new_data_pred
new_classes = new_data_pred.prediction.unique()
if domaine!='' :
    new_classes = [domaine]

for i in new_classes:
    if os.path.isfile('tfidf_' + str(i) + '.pickle') and os.path.isfile("models/concept/svc_concept_" + str(i) + ".pkl") and os.path.isfile("mlb_"+str(i)+".pickle"):
        print(str(i))
        dataset1 = new_data_pred.loc[new_data_pred['prediction'] == i]
        
        dataset = dataset1.copy()
        dataset.titre = dataset.titre.map(lambda e: word_tokenize(e, language = 'french'))
        dataset.resume = dataset.resume.map(lambda e: word_tokenize(e, language = 'french'))
        #dataset=preprocessing(dataset1, 'french', 'test')       
        # concatenate tokens
        #dataset['titre'] = dataset.titre.map(lambda e: ' '.join(e))
        dataset.titre = dataset.titre.map(lambda e: ' '.join(e))
        dataset.resume = dataset.resume.map(lambda e: ' '.join(e)) 
        dataset['corpus'] = dataset['titre']  +' ' + dataset['resume']
        #print(dataset.corpus)
        vectorizer = pickle.load(open('tfidf_' + str(i) + '.pickle', 'rb'))
        new_X = vectorizer.transform(dataset.corpus)
        model = joblib.load("models/concept/svc_concept_" + str(i) + ".pkl")
        pred = model.predict(new_X)
        mlb = pickle.load(open("mlb_"+str(i)+".pickle", 'rb'))
        y_pred_inversed = mlb.inverse_transform(pred)
        
        for item,label in zip(dataset.idB, y_pred_inversed):
            for x in label:
                #print ('  %s => %s' % (item[0:200],label))
                df_resu_ppn = df_resu_ppn.append({'idB': item,'algo': 'svc','auto':x,'indexeur':'svc'}, ignore_index=True)
                #optionnel
                
                c = conn.cursor()
                c.execute("select datas from autorites.AUT_2XX where tag='250$a' and ppn='"+x+"'")
                for row in c:
                    #print (row[0]) 
                    #print ('      %s => %s => %s' % (item[0:200],x,row[0]))
                    df_resu = df_resu.append({'idB': item,'auto':row[0]}, ignore_index=True)
                    
                c.close()
oracle_disconnect()
df_resu.sort_values(by=['idB'])
df_resu_ppn.sort_values(by=['idB'])
df_resu=df_resu.groupby(['idB'])['auto'].apply(','.join).reset_index()

df_resu.to_excel("output/prediction_concepts_newdata_aline.xlsx") 

                                 
df_resu_ppn.to_excel("evaluation_support/indexation_auto/auto.xlsx")
print(df_resu_ppn.head(10))
df_resu_ppn=df_resu_ppn.drop('algo', 1)
df_resu_ppn=df_resu_ppn.groupby(['idB','indexeur'])['auto'].apply(','.join).reset_index()
df_resu_ppn.indexeur = 'indexeur 1'
df_resu_ppn_csv=pd.concat([df_resu_ppn['idB'],df_resu_ppn['indexeur'],df_resu_ppn['auto'].str.split(',',n=20,expand=True)], axis = 1)
df_resu_ppn_csv.to_csv("evaluation_support/indexation_indexeurs/indexeur1.csv", index=False)
pd.set_option('display.width', 800)
pd.set_option('display.max_rows', None)

print(df_resu.head(10))

G300
G650
G320
G910
G200
G621
G700
G900
G400
G500
G520
G550
G510
G530
G690
G610
G560
G630
G640
G540
G600
G800
G150
G793
G580
G577
G370
G100
         idB indexeur       auto algo
0  002731401      svc  027238563  svc
1  002731401      svc  02723858X  svc
2  002882418      svc  027690342  svc
3  003510972      svc  027791246  svc
4  003888908      svc  027286460  svc
5  00457382X      svc  02722550X  svc
6  00457382X      svc  027790088  svc
7  004574230      svc  027791246  svc
8  004576675      svc  027261697  svc
9  004617452      svc  049647490  svc
         idB                                               auto
0  000217549                         Technique de la production
1  00212629X                                    Paléogéographie
2  002587432    Politique et gouvernement,Relations extérieures
3  002632489                                 Droit,Lutte contre
4  002731401                  Mariage,Mariage,Biens incorporels
5  002736098  Division internationale du travail,Politique

In [14]:
dataset_rec.head(10)

,idB
0,000217549
1,000217549
2,000217549
3,000217549
4,000217549
5,000217549
6,000217549
7,000217549
8,000217549
9,001976508


# Prédiction des concepts sur des nouvelles données

In [189]:
#recherche les domaines
#attention le 570 n a pas pu etre entrainé car pas assez de données
import requests
import xml.etree.ElementTree as ET
df = pd.DataFrame(columns=['titre','concept'])



new_data = pd.read_csv('evaluation/edp_titres_resumes_fr_1.csv')
new_data=new_data.head(10)
new_data_preprocessed = preprocessing(new_data, 'french', 'test')
new_data_preprocessed['idB'] = new_data.idB.map(lambda e: word_tokenize(e, language = 'french'))
new_data_preprocessed['origine'] = new_data.titre.astype(str)
new_data_preprocessed['titre'] = new_data_preprocessed.titre.map(lambda e: ' '.join(e))
new_data_preprocessed['resume'] = new_data_preprocessed.resume.map(lambda e: ' '.join(e)) 
new_data_preprocessed['corpus'] = new_data_preprocessed['titre'] + new_data_preprocessed['resume']
#new_data_preprocessed.corpus
model = joblib.load("models/svc_domaine.pkl")
mlb = pickle.load(open("mlb_domaine.pickle", 'rb'))
vectorizer = pickle.load(open('tfidf_domaine.pickle', 'rb'))
X_test_vec = vectorizer.transform(new_data_preprocessed.corpus)
y_pred = model.predict(X_test_vec)
#Afficher les résultats de prédiction
y_pred_inversed = mlb.inverse_transform(y_pred)
#for item,label in zip(new_data_preprocessed.corpus, y_pred_inversed):
#    print ('%s => %s' % (item[0:15],label))

    
new_data_preprocessed['prediction'] = pd.Series(y_pred_inversed, index=new_data_preprocessed.index)
new_data_pred = new_data_preprocessed.explode('prediction')
#new_data_pred
new_classes = new_data_pred.prediction.unique()

for i in new_classes:
    if os.path.isfile('tfidf_' + str(i) + '.pickle') and os.path.isfile("models/concept/svc_concept_" + str(i) + ".pkl") and os.path.isfile("mlb_"+str(i)+".pickle"):
        print(str(i))
        dataset1 = new_data_pred.loc[new_data_pred['prediction'] == i]
        dataset = dataset1.copy()
        #dataset = dataset1.copy()
        dataset.titre = dataset.titre.map(lambda e: word_tokenize(e, language = 'french'))
        dataset.resume = dataset.resume.map(lambda e: word_tokenize(e, language = 'french'))
        #dataset=preprocessing(dataset1, 'french', 'test')       
        # concatenate tokens
        dataset['titre'] = dataset.titre.map(lambda e: ' '.join(e))
        dataset['resume'] = dataset.resume.map(lambda e: ' '.join(e)) 
        dataset['corpus'] = dataset['titre']  +' ' + dataset['resume']
        #print(dataset.corpus)
        vectorizer = pickle.load(open('tfidf_' + str(i) + '.pickle', 'rb'))
        new_X = vectorizer.transform(dataset.corpus)
        model = joblib.load("models/concept/svc_concept_" + str(i) + ".pkl")
        pred = model.predict(new_X)
        mlb = pickle.load(open("mlb_"+str(i)+".pickle", 'rb'))
        y_pred_inversed = mlb.inverse_transform(pred)
        for item,label in zip(dataset.origine, y_pred_inversed):
            for x in label:
                r = requests.get('https://www.idref.fr/'+x+'.xml')
                root = ET.fromstring(r.content)
                for e in root.findall('./datafield[@tag="250"]'):
                    for f in e.findall('./subfield[@code="a"]'):
                        #print ('    %s => %s' % (item[0:200],f.text))
                        df = df.append({'titre': item,'concept':f.text}, ignore_index=True)
df.sort_values(by=['titre'])
df=df.groupby(['titre'])['concept'].apply(','.join).reset_index()
df.to_excel("output/prediction_concepts_newdata.xlsx") 
pd.set_option('display.width', 150)
pd.set_option('display.max_rows', None)
print(df)


G530
G610
G800
G320
G900
G100
G200
G510
G640
G700
G600
G650
                                               titre                                            concept
0  Diabète en prison : mise en place d’un program...                                            Prisons
1  Impact d’un programme d’éducation thérapeutiqu...  Éducation des patients,Patients,Évaluation,Pat...
2  L’autonomie du patient, d’une sémantique uniqu...                 Relations médecin-patient,Patients
3  Similitudes, différences, complémentarités ent...                                          Recherche
4  Usage du jeu vidéo pour l’éducation thérapeuti...  Éducation des patients,Emploi en thérapeutique...


In [187]:
de=df.copy()
de=de.groupby(['titre'])['concept'].apply(','.join).reset_index()
de

,titre,concept
0,Diabète en prison : mise en place d’un program...,"Diabètes,Thérapeutique,Éducation des patients"
1,"L’autonomie du patient, d’une sémantique uniqu...","Éducation des patients,Relations médecin-patie..."
